In [1]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather
from typing import Union

In [2]:
df = feather.read_feather('/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/target_df.feather')

In [3]:
sample = df[:10000]

In [4]:
sample

,user_id,items_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
...,...,...,...
9995,117,352458,130
9996,117,169011,131
9997,117,360207,132
9998,117,66925,133


In [5]:
class UsersKFoldPOut():

    def __init__(self, n_folds, user_column: str, order_column: str, random_seed: int, p: int):
        self.n_folds = n_folds
        self.user_column = user_column
        self.order_column = order_column
        self.random_seed = random_seed
        self.p = p


    def split(self, df: pd.DataFrame) -> Union[list, list]:
        """
        Method for split dataset
        :param df: your dataset
        :return: train indexes, test indexes
        """
        users = df[self.user_column].unique()
        users_count = len(users)

        np.random.seed(self.random_seed)
        np.random.shuffle(users)

        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0

        for fold_size in fold_sizes:

            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users) & (df[self.order_column] < self.p)
            train_mask = ~df[self.user_column].isin(test_fold_users)

            yield train_mask, test_mask

In [6]:
n_folds = 3
p = 5
RAND = 23

In [7]:
cv = UsersKFoldPOut(n_folds, user_column='user_id', order_column='order', random_seed=RAND, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(sample)):
    train = sample[train_mask]
    test = sample[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 6200, Test: 200
Fold#1 | Train: 6456, Test: 195
Fold#2 | Train: 6456, Test: 195


In [15]:
train[:10]

,user_id,items_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
5,0,95202,5
6,0,196872,6
7,0,352770,7
8,0,319811,8
9,0,122447,9


In [16]:
test[:10]

,user_id,items_id,order
83,3,281997,0
84,3,169130,1
85,3,429953,2
86,3,280795,3
87,3,253729,4
176,5,375102,0
177,5,214473,1
178,5,438299,2
179,5,28369,3
180,5,291548,4


In [17]:
print(f'Количество пользователей из train в test: {np.sum(train["user_id"].isin(test["user_id"]))}')

Количество пользователей из train в test: 0
